# Modeling GNNePCSAFT


### Load env variables


In [ ]:
import os

os.environ["WANDB_API_KEY"] = ""  # Your wandb api key

### Git clone repo


In [ ]:
%%bash

# for the cloud 
ls -a
rm -rf .* *
git clone --branch dev --single-branch -q https://github.com/wildsonbbl/gnnepcsaft.git .

### Pip install requirements


In [ ]:
%%bash

## uv pip install -r requirements-torch.txt # if torch not installed

uv pip install -r requirements.txt

echo "*********  Finished pip installing requirements.txt  *********"



### Check feos pkg, cpu and gpu


In [ ]:
%%bash

python --version

pip show feos | grep -E "Version|Name"

pip show torch | grep -E "Version|Name"

lscpu | grep -E "Model name|Core|core" 

nvidia-smi

### Training run


In [ ]:
!python -m gnnepcsaft.train.train \
--workdir=$PWD/gnnepcsaft --config=$PWD/gnnepcsaft/configs/default.py \
--framework="lightning"  \
--num_cpu=1.0 --num_gpus=0.333 --num_workers=3 \
--config.accelerator=auto


In [ ]:
%%bash

echo "*****  Finished training  *****"